In [ ]:
# TTS and audio imports
from TTS.api import TTS
from pydub import AudioSegment
# import wave
import simpleaudio as sa
import librosa
import pyrubberband
import soundfile as sf

# Progress bar imports
from tqdm.auto import tqdm

# GUI, keyboard and clipboard imports
import pyautogui as pya
import pyperclip
from pynput import keyboard

# System imports
from sys import platform
import time
import threading
import textwrap
import re

In [ ]:
# TTS.list_models()[8]

In [ ]:
# load model on GPU if possible if not CPU
try:
    tts = TTS('tts_models/en/ljspeech/tacotron2-DDC_ph',gpu=True)
except:
    tts = TTS('tts_models/en/ljspeech/tacotron2-DDC_ph')

In [ ]:
def split_text(input_text, max_length):
    # Split the input text on periods and commas
    smaller_chunks = re.split(r'[.,]', input_text)
    # apply the textwrap.wrap function to each chunk
    smaller_chunks = [textwrap.wrap(chunk, max_length, break_on_hyphens=True, break_long_words=True) for chunk in smaller_chunks]
    
    # Flatten the list of smaller chunks and append a period or comma to the end of each chunk that does not already end with a period or comma
    smaller_chunks = [item + (item[-1] not in '.,' and '. ' or ' ') for sublist in smaller_chunks for item in sublist] 
            
    # Return the list of smaller chunks
    return smaller_chunks

In [ ]:
# Generate an audio file for the specified text and save it to the specified file
def run_tts(token,filename):
    # Use the TTS API to generate an audio file for the text
    tts.tts_to_file(text=token,
                    speaker=tts.speakers,
                    language=tts.languages,
                    file_path=filename)

In [ ]:
# speed up audio track with pyrubberband
def speed_up(speed,file_name):
    y, sr = librosa.load(file_name, sr=None)
    y_stretched = pyrubberband.time_stretch(y, sr, speed)
    sf.write(file_name, y_stretched, sr, format='wav')

In [ ]:
def build_audio(token,speed):
    # Set the file name for the audio file
    file_name = "TTS_next.wav"
    
    # Generate an audio file for the current token
    run_tts(token, file_name)

    # Use the AudioSegment class to load the audio file
    audio = AudioSegment.from_file(file_name)
    
    # Trim the audio file
    try:
        audio = audio[30:-350]
        
        # Save the modified audio to a file
        audio.export(file_name, format = 'wav')
        
        if speed > 1:
            speed_up(speed,file_name)

    except:
        print('Small audio track')
    
    # Load the audio file
    wave_obj = sa.WaveObject.from_wave_file(file_name)
    
    return wave_obj

In [ ]:
def read_text(input_text, speed, stop_flag):
    # Split the input text into a list of tokens with a maximum length of 200 characters
    tokens = split_text(input_text, 300)
    
    # Iterate over each token
    for token in tokens:
       
        wave_obj = build_audio(token,speed)

        # If there is an audio thread currently playing, wait until it finishes before playing the new audio
        if 'audio_thread' in locals():
            while audio_thread.is_playing():
                time.sleep(0.1)
                # Check the stop flag
                if stop_flag.is_set():
                    audio_thread.stop()
                    
        # Check the stop flag
        if stop_flag.is_set():
            # Break out of the loop if the stop flag is set
            break
        
        # Play the audio file
        audio_thread = wave_obj.play()

In [ ]:
# Create the stop flag
stop_flag = threading.Event()
reading_thread = threading.Thread()

In [ ]:
# Set the stop flag to stop the read_text function
stop_flag.set()

In [ ]:
def copy_clipboard():
#     empty clipboard
    pyperclip.copy('')
#     copy new text
    if platform == "darwin":
        pya.hotkey('command', 'c')
    else:
        pya.hotkey('ctrl', 'c')
        
#     wait for the cliboard to change 
    max_wait_time = 0.5
    wait_start_time = time.time_ns()
    current_wait_time = 0
    while max_wait_time < current_wait_time:
        if pyperclip.paste() != '':
            break
        time.sleep(.01)
        current_wait_time = time.time_ns() - wait_start_time
        print(current_wait_time)
    
    return pyperclip.paste()

In [ ]:
def start_reading(speed):
    
    global stop_flag
    global reading_thread
    
    if reading_thread.is_alive():
        stop_flag.set()
    else:
        stop_flag = threading.Event()
        reading_thread = threading.Thread(target=read_text,args=[copy_clipboard(),speed,stop_flag])
        reading_thread.start()

In [ ]:
def on_press(key):
    global reading_thread
#     16777215 is kapslock
    if key == keyboard.KeyCode(16777215):
        start_reading(2)
    
    if key == keyboard.Key.esc:
        print('STOPPED')
        # Stop listener
        return False


In [ ]:
listener = keyboard.Listener(on_press=on_press)
listener.start()